In [143]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

In [25]:
books = pd.read_csv('data/Books.csv')
users = pd.read_csv('data/Users.csv')
ratings = pd.read_csv('data/Ratings.csv')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_20152\927319382.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/Books.csv')


In [ ]:
ratings_with_name = ratings.merge(books, on='ISBN')

In [66]:
num_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].count().reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)

num_rating_df.head()

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [67]:
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)

avg_rating_df.head()

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [70]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

popular_df.head()

,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453


In [86]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index

filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [87]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

Item-Based Collaborative Filtering

In [243]:
pt_item = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt_item.fillna(0, inplace=True)

pt_item.iloc[:, :5].head(15)

User-ID,254,2276,2766,2977,3363
Book-Title,,,,,
1984,9.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0
A Case of Need,0.0,0.0,0.0,0.0,0.0
"A Child Called \It\"": One Child's Courage to Survive""",0.0,0.0,0.0,0.0,0.0
A Civil Action,0.0,0.0,0.0,0.0,0.0
A Day Late and a Dollar Short,0.0,0.0,0.0,0.0,0.0


In [ ]:
item_similarity = cosine_similarity(pt_item)

def recommend_item_based(book_name):
    index = np.where(pt_item.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(item_similarity[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt_item.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [ ]:
recommend_item_based('You Belong To Me')

[['Loves Music, Loves to Dance',
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0671758896.01.MZZZZZZZ.jpg'],
 ["I'll Be Seeing You",
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0671888587.01.MZZZZZZZ.jpg'],
 ['Before I Say Good-Bye',
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0671004573.01.MZZZZZZZ.jpg'],
 ["Daddy's Little Girl",
  'Mary Higgins Clark',
  'http://images.amazon.com/images/P/0743206045.01.MZZZZZZZ.jpg']]

User-Based Collaborative Filtering

In [242]:
pt_user = final_ratings.pivot_table(index='User-ID', columns='Book-Title', values='Book-Rating')
pt_user.fillna(0, inplace=True)

pt_user.iloc[:, :5].head(15)

Book-Title,1984,1st to Die: A Novel,2nd Chance,4 Blondes,A Bend in the Road
User-ID,,,,,
254,9.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,10.0,0.0,0.0
2766,0.0,0.0,0.0,0.0,7.0
2977,0.0,0.0,0.0,0.0,0.0
3363,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0
6251,0.0,0.0,0.0,0.0,0.0
6323,0.0,0.0,0.0,0.0,0.0


In [240]:
user_similarity = cosine_similarity(pt_user)

def recommend_user_based(user_id):
    if user_id not in pt_user.index:
        return f"User {user_id} không tồn tại!"
    
    index = np.where(pt_user.index == user_id)[0][0]
    similar_users = sorted(list(enumerate(user_similarity[index])),key=lambda x:x[1],reverse=True)[1:5]

    similar_user_ids = [pt_user.index[i] for i, _ in similar_users]
    similar_users_ratings = pt_user.loc[similar_user_ids]

    user_books = pt_user.loc[user_id]
    books_read = user_books[user_books > 0].index

    mean_ratings = similar_users_ratings.mean(axis=0)
    mean_ratings = mean_ratings.drop(books_read, errors="ignore")

    top_books = mean_ratings.sort_values(ascending=False).head(5)

    results = []
    for title in top_books.index:
        book_info = books[books["Book-Title"] == title][
            ["Book-Title", "Book-Author", "Image-URL-M"]
        ].drop_duplicates().values.tolist()
    
        if book_info:
            results.extend(book_info)   

    return results

In [241]:
recommend_user_based(278418)

[['Key of Knowledge (Key Trilogy (Paperback))',
  'Nora Roberts',
  'http://images.amazon.com/images/P/0515136379.01.MZZZZZZZ.jpg'],
 ['Inner Harbor (Quinn Brothers (Paperback))',
  'Nora Roberts',
  'http://images.amazon.com/images/P/0515124214.01.MZZZZZZZ.jpg'],
 ['Key of Light (Key Trilogy (Paperback))',
  'Nora Roberts',
  'http://images.amazon.com/images/P/051513628X.01.MZZZZZZZ.jpg'],
 ['Portrait in Death',
  'Nora Roberts',
  'http://images.amazon.com/images/P/0425189031.01.MZZZZZZZ.jpg'],
 ['Four To Score (A Stephanie Plum Novel)',
  'Janet Evanovich',
  'http://images.amazon.com/images/P/0312966970.01.MZZZZZZZ.jpg']]

In [107]:
books.drop_duplicates('Book-Title')

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271354,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...
